In [1]:
import plotly

ModuleNotFoundError: No module named 'plotly'

In [7]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from scipy.stats import gaussian_kde

# Example DataFrame
data = {'GP': [
582,
509,
581,
538,
552,
558,
496,
535,
549,
539,
477,
541,
506,
512,
501,
469,
               ]}  # Skewed distribution
df = pd.DataFrame(data)

# Column to visualize
column = 'GP'
custom_x = 539

# Kernel Density Estimate (KDE) for smoothing the distribution
kde = gaussian_kde(df[column])
x_vals = np.linspace(df[column].min(), df[column].max(), 500)
y_vals = kde(x_vals)

# Create the Plotly figure
fig = go.Figure()

# Add the KDE line (smoothed curve)
fig.add_trace(go.Scatter(
    x=x_vals,
    y=y_vals,
    mode='lines',
    line=dict(color='black', width=3),
    showlegend=False
))

fig.add_trace(go.Scatter(
    x=[custom_x, custom_x],
    y=[min(y_vals), max(y_vals)],
    mode='lines',
    line=dict(color='black', width=2, dash='dash'),
    showlegend=False
))

# Add a label for the custom x value
fig.add_trace(go.Scatter(
    x=[custom_x],
    y=[max(y_vals) * 1.03],  # Position the label 80% up the y-axis for visibility
    text=[f"{custom_x}"],
    mode='text',
    textfont=dict(color='black', size=20),
    showlegend=False
))

# Update layout for a minimalist look
fig.update_layout(
    xaxis=dict(showline=False, showgrid=False, zeroline=False, title='', visible=False),
    yaxis=dict(showline=False, showgrid=False, zeroline=False, title='', visible=False),
    template='plotly_white',
    margin=dict(l=10, r=10, t=10, b=10)
)

# Show the plot
fig.show()

In [11]:
def create_plot(data, data_point=None):
    column = 'data'
    data = {column: data}  # Skewed distribution
    df = pd.DataFrame(data)

    # Kernel Density Estimate (KDE) for smoothing the distribution
    kde = gaussian_kde(df[column])
    x_vals = np.linspace(df[column].min(), df[column].max(), 500)
    y_vals = kde(x_vals)

    # Create the Plotly figure
    fig = go.Figure()

    # Add the KDE line (smoothed curve)
    fig.add_trace(go.Scatter(
        x=x_vals,
        y=y_vals,
        mode='lines',
        line=dict(color='black', width=3),
        showlegend=False
    ))

    fig.add_trace(go.Scatter(
        x=[data_point, data_point],
        y=[min(y_vals), max(y_vals)],
        mode='lines',
        line=dict(color='black', width=2, dash='dash'),
        showlegend=False
    ))

    # Add a label for the custom x value
    fig.add_trace(go.Scatter(
        x=[data_point],
        y=[max(y_vals) * 1.03],  # Position the label 80% up the y-axis for visibility
        text=[f"{data_point}"],
        mode='text',
        textfont=dict(color='black', size=20),
        showlegend=False
    ))

    # Update layout for a minimalist look
    fig.update_layout(
        xaxis=dict(showline=False, showgrid=False, zeroline=False, title='', visible=False),
        yaxis=dict(showline=False, showgrid=False, zeroline=False, title='', visible=False),
        template='plotly_white',
        margin=dict(l=10, r=10, t=10, b=10)
    )
    return fig

In [14]:
create_plot([
582,
509,
581,
538,
552,
558,
496,
535,
549,
539,
477,
541,
506,
512,
501,
469,
               ], 509.0)

In [ ]:
import altair as alt
import pandas as pd
import numpy as np

df = pd.read_csv(r"stats_detail.csv")
df_grouped = df.groupby(['value_dt', 'pooler_name'])['total_points'].sum().reset_index()

# Create a selection that chooses the nearest point & selects based on x-value
nearest = alt.selection_point(nearest=True, on="pointerover",
                              fields=["value_dt"], empty=False)

# The basic line
line = alt.Chart(df_grouped).mark_line(interpolate="basis").encode(
    x='value_dt:T',  # Date on the X-axis
    y='total_points:Q',  # Total points on the Y-axis
    color='pooler_name:N',  # Different lines for each pooler_name
).properties(
    title="Evolution of Total Points per Pooler"
)

# Put the five layers into a chart and bind the data
alt.layer(
    line
).properties(
    width=600, height=300
)

In [12]:
import altair as alt
import pandas as pd
import streamlit as st
import datetime as dt

df = pd.read_csv(r"stats_detail.csv")
df['value_dt'] = pd.to_datetime(df['value_dt'])
df['day'] = (df['value_dt'] - dt.datetime(2024, 9, 30)).dt.days
max_day = df['day'].max()
#df = df[df['day'] >= max_day - 31]
df_grouped = df.groupby(['day', 'pooler_name'])['total_points'].sum().reset_index()
df_grouped['normalized_points'] = df_grouped['total_points'] - df_grouped.groupby('day')['total_points'].transform('median')
source = df_grouped[['pooler_name', 'day', 'total_points', 'normalized_points']]
max_y = source['normalized_points'].max() + 10
min_y = source['normalized_points'].min() - 10

# Create the line chart using Altair
line = alt.Chart(source).mark_line(interpolate="basis").encode(
    x='day:Q',  # Date on the X-axis
    y=alt.Y('normalized_points:Q', scale=alt.Scale(domain=[min_y, max_y])),  # Total points on the Y-axis
    color='pooler_name:N',  # Different lines for each pooler_name
)

# Put the five layers into a chart and bind the data
alt.layer(
    line
).properties(
    width=600, height=300
)

alt.LayerChart(...)

In [34]:
import plotly.graph_objects as go
import numpy as np
import pandas as pd

df = pd.read_csv(r"stats_detail.csv")
df['value_dt'] = pd.to_datetime(df['value_dt'])
df['day'] = (df['value_dt'] - dt.datetime(2024, 9, 30)).dt.days
max_day = df['day'].max()
#df = df[df['day'] >= max_day - 31]
df_grouped = df.groupby(['day', 'pooler_name'])['total_points'].sum().reset_index()
df_grouped['normalized_points'] = df_grouped['total_points'] - df_grouped.groupby('day')['total_points'].transform('median')
source = df_grouped[['pooler_name', 'day', 'total_points', 'normalized_points']]
max_y = source['normalized_points'].max() + 10
min_y = source['normalized_points'].min() - 10

# Generate sample data
x = np.linspace(0, 10, 100)
y = np.sin(x) + np.random.normal(0, 0.2, size=x.shape)

# Apply smoothing (simple moving average in this case)
window_size = 5
y_smoothed = np.convolve(y, np.ones(window_size)/window_size, mode='valid')

# Create the plot
fig = go.Figure()

# Add the original line
fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name='Original', line=dict(color='blue')))

# Add the smoothed line
fig.add_trace(go.Scatter(x=x[window_size-1:], y=y_smoothed, mode='lines', name='Smoothed', line=dict(color='red', dash='dash')))

# Update layout
fig.update_layout(title="Line Chart with Smoothed Line",
                  xaxis_title="X Axis",
                  yaxis_title="Y Axis")

# Show the plot
fig.show()

In [127]:
import datetime as dt
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from scipy.interpolate import CubicSpline
import plotly.express as px
from PIL import Image
from src.style import team_colors
from plotly.graph_objects import Layout

df = pd.read_csv(r"stats_detail.csv")
df['value_dt'] = pd.to_datetime(df['value_dt'])
df['day'] = (df['value_dt'] - dt.datetime(2024, 9, 30)).dt.days
max_day = df['day'].max()
#df = df[df['day'] >= max_day - 31]
df_grouped = df.groupby(['day', 'pooler_name', 'pooler_team'])['total_points'].sum().reset_index()
df_grouped['normalized_points'] = df_grouped['total_points'] - df_grouped.groupby('day')['total_points'].transform('median')
df = df_grouped[['pooler_name', 'pooler_team', 'day', 'total_points', 'normalized_points']]
df = df.rename(columns={
    'day': 'x',
    'normalized_points': 'y'
})
max_y = df['y'].max() + 10
min_y = df['y'].min() - 10

# Use a color palette from Plotly
colors = px.colors.qualitative.Set1  # You can choose other palettes like 'Set2', 'Pastel1', etc.

# Create the plot
layout = Layout(plot_bgcolor='rgba(255,255,255,1)')
fig = go.Figure(layout=layout)


# Loop over each unique pooler_name to generate a cubic spline curve
for i, r in df[df['x']==max_day].sort_values('y', ascending=True).iterrows():
    # Get data for the specific pooler_name
    pooler_name = r["pooler_name"]
    df_pooler = df[df['pooler_name'] == pooler_name]

    pooler_team = r['pooler_team']
    team_logo = f"assets/img/teams/{pooler_team}.png"
    team_color = team_colors[pooler_team]
    color = f"rgba({','.join([str(c) for c in team_color.primary.rgb])}, 0.7)"


    # Create the cubic spline interpolation
    cs = CubicSpline(df_pooler['x'], df_pooler['y'])

    # Generate smooth x values for the spline curve
    x_smooth = np.linspace(df_pooler['x'].min(), df_pooler['x'].max(), 1000)
    y_smooth = cs(x_smooth)

    # Get a color from the palette based on the index
    #color = "rgba(0, 104, 71, 0.7)" # Ensure color index stays within range

    # Add the smooth cubic spline curve for the current pooler_name (no markers)
    fig.add_trace(go.Scatter(
        x=x_smooth,
        y=y_smooth,
        mode='lines',
        name=f'{pooler_name}',
        line=dict(color=color, dash='solid', width=5),
        hovertemplate=f'{pooler_name}'
    ))


    pooler_initials = ''.join([word[0] for word in pooler_name.split()])
    fig.add_trace(go.Scatter(x=[df_pooler['x'].iloc[-1]], y=[df_pooler['y'].iloc[-1]],
                             mode='markers+text', name=f'{pooler_name}',
                             marker=dict(color=color, size=30),
                             hoverinfo='none'))  # Set font color to match the marker

    # Add the marker only at the last point for the current pooler_name
    fig.add_trace(go.Scatter(x=[df_pooler['x'].iloc[-1]], y=[df_pooler['y'].iloc[-1]],
                             mode='markers+text', name=f'{pooler_name}',
                             marker=dict(color="rgba(255,255,255, 0.7)", size=27),
                             hoverinfo='none'))  # Set font color to match the marker



    print([df_pooler['x'].iloc[-1]], [df_pooler['y'].iloc[-1]])
    fig.add_layout_image(
        dict(
            source=team_logo,  # Replace with your image URL
            x=df_pooler['x'].iloc[-1],
            y=df_pooler['y'].iloc[-1],
            xref="x",  # Use the x-axis for reference
            yref="y",  # Use the y-axis for reference
            xanchor="center",  # Center the image horizontally
            yanchor="middle",  # Center the image vertically
            sizex=12,  # Image width in data coordinates
            sizey=12,  # Image height in data coordinates
            opacity=0.6  # Set image transparency
        )
    )

# Update layout
fig.update_layout(showlegend=False,
                  width=700,
                  height=600,
                  margin=dict(t=25, l=25, b=25, r=25),
                  xaxis=dict(visible=False, fixedrange=True),  # Remove x-axis grid and labels
                  yaxis=dict(visible=False, fixedrange=True ) # Keep y-axis grid lines
)

# Show the plot
fig.show()

[np.int64(116)] [np.float64(-113.5)]
[np.int64(116)] [np.float64(-87.5)]
[np.int64(116)] [np.float64(-54.5)]
[np.int64(116)] [np.float64(-35.5)]
[np.int64(116)] [np.float64(-31.5)]
[np.int64(116)] [np.float64(-20.5)]
[np.int64(116)] [np.float64(-2.5)]
[np.int64(116)] [np.float64(-0.5)]
[np.int64(116)] [np.float64(0.5)]
[np.int64(116)] [np.float64(8.5)]
[np.int64(116)] [np.float64(9.5)]
[np.int64(116)] [np.float64(24.5)]
[np.int64(116)] [np.float64(41.5)]
[np.int64(116)] [np.float64(69.5)]
[np.int64(116)] [np.float64(73.5)]
[np.int64(116)] [np.float64(97.5)]


In [112]:
import plotly.graph_objects as go

# Create a scatter plot
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=[1, 2, 3],  # X-coordinates of the points
    y=[4, 5, 6],  # Y-coordinates of the points
    mode='markers',
    marker=dict(size=10, color='blue'),
    name='Data Points'
))

# Add an image relative to data coordinates
fig.add_layout_image(
    dict(
        source="https://upload.wikimedia.org/wikipedia/commons/thumb/a/a9/Example.jpg/120px-Example.jpg",  # Replace with your image URL
        x=2,  # X-coordinate of the image
        y=5,  # Y-coordinate of the image
        xref="x",  # Image moves relative to the x-axis
        yref="y",  # Image moves relative to the y-axis
        xanchor="center",  # Center the image horizontally
        yanchor="middle",  # Center the image vertically
        sizex=0.5,  # Width of the image in data coordinates
        sizey=0.5,  # Height of the image in data coordinates
        opacity=1  # Fully opaque
    )
)

# Customize the layout for zoom interactivity
fig.update_layout(
    title="Scatter Plot with Image Fully Relative to Zoom",
    xaxis=dict(
        title="X-Axis",
        range=[0, 4],  # Initial x-axis range
        fixedrange=True  # Allow zooming and panning
    ),
    yaxis=dict(
        title="Y-Axis",
        range=[3, 7],  # Initial y-axis range
        fixedrange=True  # Allow zooming and panning
    ),
    dragmode="zoom",  # Enable zooming and panning
    hovermode="closest",  # Enable closest point highlighting on hover
)

fig.show()
